# Login / Registration Page Connected to PostgreSQL Database

In [1]:
from tkinter import *
import os
import psycopg2
from passlib.hash import sha256_crypt
import datetime
import time
import pandas as pd
from pandas import DataFrame

/Users/bsnelson/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def delete3():
    screen3.destroy()
    screen2.destroy()
def delete4():
    screen4.destroy()
def delete5():
    screen5.destroy()
def delete6():
    screen6.destroy()

In [3]:
def login_success():
    global screen3
    screen3 = Toplevel(screen)
    screen3.title("Success")
    screen3.geometry("150x100")
    Label(screen3, text = "Login Success").pack()
    Button(screen3, text = "OK", command = delete3).pack()

In [4]:
def password_not_recognised():
    global screen4
    screen4 = Toplevel(screen)
    screen4.title("Error")
    screen4.geometry("150x100")
    Label(screen4, text = "Incorrect Password").pack()
    Button(screen4, text = "OK", command = delete4).pack()

In [5]:
def email_not_found():
    global screen5
    screen5 = Toplevel(screen)
    screen5.title("Error")
    screen5.geometry("150x100")
    Label(screen5, text = "Incorrect Email").pack()
    Button(screen5, text = "OK", command = delete5).pack()

In [6]:
def email_duplicate():
    global screen6
    screen6 = Toplevel(screen)
    screen6.title("Error")
    screen6.geometry("200x150")
    Label(screen6, text = "Email Already Registered").pack()
    Button(screen6, text = "Try Again", command = delete6).pack()

In [7]:
def update_data_base():
    con = psycopg2.connect(database="Login",user = "postgres", password = "****")
    cur = con.cursor()
    
    cur.execute("SELECT * from users")
    global mydf
    mydf = DataFrame(cur.fetchall())
    
    mydf.columns = ['id','first_name','last_name','email','password','time_stamp']
    
    con.commit()

    cur.close()
    con.close()

update_data_base()

In [8]:
def get_email_index(the_df, the_email):
    for index, row in the_df.iterrows():
        if the_email in row['email']:
            return index

In [9]:
def login_verify():
    email1 = email_verify.get()
    password1 = password_verify.get()
    

  #  sha256_crypt.verify("password", password1)
    if email1 in mydf['email'].tolist():
        index = get_email_index(mydf, email1)
        y = mydf['password'][index]
        if sha256_crypt.verify(password1, y):
            login_success()
        else:
            password_not_recognised()
    else: 
        email_not_found()
    
    email_entry1.delete(0, END)
    password_entry1.delete(0, END)
    

In [10]:
def login():
    global screen2
    screen2 = Toplevel(screen)
    screen2.title("Login")
    screen2.geometry("300x250")
    Label(screen2, text = "Please enter Login details below").pack()
    Label(screen2, text = "").pack()
    
    global email_verify
    global password_verify
    
    email_verify = StringVar()
    password_verify = StringVar()
    
    global email_entry1
    global password_entry1 
    
    Label(screen2, text="Email * ").pack()
    email_entry1 = Entry(screen2, textvariable = email_verify)
    email_entry1.pack()
    Label(screen2, text="").pack()
    Label(screen2, text="Password * ").pack()
    password_entry1 = Entry(screen2, textvariable = password_verify, show="*")
    password_entry1.pack()
    Label(screen2, text="").pack()
    Button(screen2, text = "Login", width = 10, height = 1, command = login_verify).pack()
    

In [11]:
def register_user():
    
    update_data_base()
    global mydf
    email_info = email.get()
    password_info = password.get()
    first_name_info = first_name.get()
    last_name_info = last_name.get()
    

    if email_info in mydf['email'].tolist():
        email_duplicate()
        return
    
    
    hash_pas = sha256_crypt.encrypt(password_info)
    
    ts = time.time()
    current_time = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    

    
    con = psycopg2.connect(database="Login",
                      user = "postgres",
                      password = "*****")

    cur = con.cursor()

    cur.execute("insert into users (first_name, last_name, email, password, activation_date) values (%s, %s, %s, %s, %s)", 
                (first_name_info, last_name_info, email_info, hash_pas, current_time))
    
    

    con.commit()

    cur.close()
    con.close()
    
    email_entry.delete(0, END)
    password_entry.delete(0, END)
    first_name_entry.delete(0,END)
    last_name_entry.delete(0, END)
    
    update_data_base()
    
    Label(screen1, text = "Registration Success", fg = "green", font = ("calibri", 11)).pack()
    

In [12]:
def register():
    global screen1
    screen1 = Toplevel(screen)
    screen1.title("Register")
    screen1.geometry("450x400")
    
    global email
    global password
    global first_name
    global last_name
    
    global email_entry
    global password_entry
    global first_name_entry
    global last_name_entry

    
    
    email = StringVar()
    password = StringVar()
    first_name = StringVar()
    last_name = StringVar()
    
    Label(screen1, text = "Please enter details below").pack()
    Label(screen1, text = "").pack()
    
    Label(screen1, text = "Email * ").pack()
    email_entry = Entry(screen1, textvariable = email)
    email_entry.pack()
    
    Label(screen1, text = "Password * ").pack()
    password_entry = Entry(screen1, textvariable = password, show="*")
    password_entry.pack()
    
    Label(screen1, text = "First Name * ").pack()
    first_name_entry = Entry(screen1, textvariable = first_name)
    first_name_entry.pack()
    
    Label(screen1, text = "Last Name * ").pack()
    last_name_entry = Entry(screen1, textvariable = last_name)
    last_name_entry.pack()
    Label(screen1, text = "").pack()
    
    
    Button(screen1, text = "Register", width = 10, height = 1, command = register_user).pack()
    

In [13]:
def main_screen():
    global screen
    screen = Tk()
    screen.geometry("3500x700")
    
    background_image = PhotoImage(file='download.png')
    background_label = Label(screen, image=background_image)
    background_label.place(relwidth=1,relheight=1)
    
    screen.title("Main")
    Label(text="Welcome!", bg="grey",width="300", height = "2", font=("Calibri",13)).pack()
    Label(text="").pack()
    Button(text="Login", width="30", height = "2", command = login).pack()
    Label(text="").pack()
    Button(text="Register", width="30", height = "2", command = register).pack()
    
    update_data_base()
    screen.mainloop()
main_screen()